In [2]:
import levit 
import levit_c 
import torch 
import tensorrt as trt 
from cuda import cudart
import numpy as np
# import pycuda.driver
# import pycuda.autoinit


In [16]:
my_model_128 = levit.LeViT_128(pretrained=True,distillation=True)
my_model_192 = levit.LeViT_192(pretrained=True,distillation=True)
my_model_128S = levit.LeViT_128S(pretrained=True,distillation=True)
my_model_256 = levit.LeViT_256(pretrained=True,distillation=True)
my_model_384 = levit.LeViT_384(pretrained=True,distillation=True)

input_tensor = torch.randn(16,3,224,224)
torch.onnx.export(my_model_128,               # model being run
                  input_tensor,                         # model input (or a tuple for multiple inputs)
                  "levit_128_onnx.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True, 
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})  # whether to execute constant folding for optimization)
torch.onnx.export(my_model_192,               # model being run
                  input_tensor,                         # model input (or a tuple for multiple inputs)
                  "levit_192_onnx.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True, 
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})  # whether to execute constant folding for optimization)
torch.onnx.export(my_model_128S,               # model being run
                  input_tensor,                         # model input (or a tuple for multiple inputs)
                  "levit_128S_onnx.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True, 
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})  # whether to execute constant folding for optimization)
torch.onnx.export(my_model_256,               # model being run
                  input_tensor,                         # model input (or a tuple for multiple inputs)
                  "levit_256_onnx.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True, 
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})  # whether to execute constant folding for optimization)
torch.onnx.export(my_model_384,               # model being run
                  input_tensor,                         # model input (or a tuple for multiple inputs)
                  "levit_384_onnx.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True, 
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})  # whether to execute constant folding for optimization)

In [21]:
def setup_engine(max_batch_size = 512,
                 max_workspace_size_n = 4,
                 onnx_path = "levit_128_onnx.onnx",
                 trtfile = "my_model.plan",
                 min_shape = (2,3,224,224),
                 common_shape = (4,3,224,224),
                 max_shape = (16,3,224,224)
                 ):
    logger = trt.Logger(trt.Logger.VERBOSE)
    builder = trt.Builder(logger)
    builder.max_batch_size = max_batch_size

    profile = builder.create_optimization_profile()

    config = builder.create_builder_config()
    config.max_workspace_size = max_workspace_size_n<<30
    config.set_flag(trt.BuilderFlag.FP16)

    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

    parser = trt.OnnxParser(network,logger)
    with open(onnx_path, "rb") as model:
        parser.parse(model.read())

    input_tensor = network.get_input(0)
    profile.set_shape(input_tensor.name,min_shape,common_shape,max_shape)
    config.add_optimization_profile(profile)

    engineString = builder.build_serialized_network(network,config)
    if(engineString == None):
        print("Fail building")
        return 
    print("Success !") 
    with open(trtfile,'wb') as f:
        f.write(engineString)
    return engineString, logger
    

In [22]:
setup_engine()

/tmp/ipykernel_24356/2388561028.py:16: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = max_workspace_size_n<<30


[05/24/2022-15:21:28] [TRT] [I] [MemUsageChange] Init CUDA: CPU +192, GPU +0, now: CPU 3388, GPU 913 (MiB)
[05/24/2022-15:21:29] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 3407 MiB, GPU 913 MiB
[05/24/2022-15:21:29] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 3415 MiB, GPU 913 MiB
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::GridAnchor_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::GridAnchorRect_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::NMS_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::Reorg_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::Region_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::Clip_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::LReLU_TRT version 1
[05/24/2022-15:21:29] [TRT] [V] Registered plugin creator - ::PriorBo

(<tensorrt.tensorrt.IHostMemory at 0x7fe0773d1d70>,
 <tensorrt.tensorrt.Logger at 0x7fe0773eb8b0>)

In [23]:
def load_engine(engine_path):
    with open(engine_path,"rb") as f:
        engine_data = f.read()
    engine = trt.Runtime(trt.Logger(trt.Logger.VERBOSE)).deserialize_cuda_engine(engine_data)
    return engine

In [24]:
my_engine = load_engine("my_model.plan")

[05/24/2022-15:23:49] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.

[05/24/2022-15:23:49] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 4202, GPU 1165 (MiB)
[05/24/2022-15:23:49] [TRT] [I] Loaded engine size: 38 MiB
[05/24/2022-15:23:49] [TRT] [V] Using cublas as a tactic source
[05/24/2022-15:23:49] [TRT] [W] TensorRT was linked against cuBLAS/cuBLAS LT 11.8.0 but loaded cuBLAS/cuBLAS LT 11.5.1
[05/24/2022-15:23:49] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +10, now: CPU 4214, GPU 1215 (MiB)
[05/24/2022-15:23:49] [TRT] [V] Deserialization required 63888 microseconds.
[05/24/2022-15:23:49] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +36, now: CPU 0, GPU 36 (MiB)


In [31]:
def allocate_buffers(engine, batch_size, data_type):
   context = engine.create_execution_context()
   context.set_binding_shape(0,[batch_size,3,224,224])
   # Determine dimensions and create page-locked memory buffers (which won't be swapped to disk) to hold host inputs/outputs.
   h_input_1 = pycuda.driver.pagelocked_empty(trt.volume(context.get_binding_shape(0)), dtype=trt.nptype(data_type))
   h_output = pycuda.driver.pagelocked_empty(trt.volume(context.get_binding_shape(1)), dtype=trt.nptype(data_type))
    # Allocate device memory for inputs and outputs.
   d_input_1 = pycuda.driver.mem_alloc(h_input_1.nbytes)

   d_output = pycuda.driver.mem_alloc(h_output.nbytes)
    # Create a stream in which to copy inputs/outputs and run inference.
   stream = pycuda.driver.Stream()
   return context,h_input_1, d_input_1, h_output, d_output, stream 

def load_images_to_buffer(pics, pagelocked_buffer):
   preprocessed = np.asarray(pics).ravel()
   np.copyto(pagelocked_buffer, preprocessed) 

def do_inference(context, pics_1, h_input_1, d_input_1, h_output, d_output, stream, batch_size, width):

   load_images_to_buffer(pics_1, h_input_1)
       # Transfer input data to the GPU.
   pycuda.driver.memcpy_htod_async(d_input_1, h_input_1, stream)

       # Run inference.
       # context.profiler = trt.Profiler()
   context.execute(batch_size, bindings=[int(d_input_1), int(d_output)])
       # Transfer predictions back from the GPU.
   pycuda.driver.memcpy_dtoh_async(h_output, d_output, stream)
       # Synchronize the stream
   stream.synchronize()
       # Return the host output.
   out = h_output.reshape((batch_size, width))
       # print(type(out))
   return out

In [25]:
def inference(engine,batch_size,input_data):
    context = engine.create_execution_context()
    context.set_binding_shape(0,[batch_size, 3, 224,224])

    _, stream = cudart.cudaStreamCreate()
    inputHost = np.ascontiguousarray(input_data.reshape(-1))
    outputHost = np.empty(context.get_binding_shape(1),dtype = trt.nptype(engine.get_binding_dtype(1)))

    _, inputDevice = cudart.cudaMallocAsync(inputHost.nbytes,stream)
    _, outputDevice = cudart.cudaMallocAsync(outputHost.nbytes,stream)

    
    cudart.cudaMemcpyAsync(inputDevice,inputHost.ctypes.data,inputHost.nbytes,cudart.cudaMemcpyKind.cudaMemcpyHostToDevice,stream)
    
    context.execute_async_v2([int(inputDevice),int(outputDevice)],stream)

    cudart.cudaMemcpyAsync(outputHost.ctypes.data,outputDevice,outputHost.nbytes,cudart.cudaMemcpyKind.cudaMemcpyDeviceToHost,stream)
    cudart.cudaStreamSynchronize(stream)

    cudart.cudaStreamDestroy(stream)
    cudart.cudaFree(inputDevice)
    cudart.cudaFree(outputDevice)
    return outputHost

In [8]:
context,inputHost, inputDevice, outputHost, outputDevice,stream = allocate_buffers(my_engine,4,trt.float32)
out = do_inference(context, np.random.randn(4,3,224,224), inputHost, inputDevice, outputHost, outputDevice, stream, 4,1000)

NameError: name 'allocate_buffers' is not defined

In [26]:
my_model = levit.LeViT_128(pretrained=True)
input_data = np.random.rand(4,3,224,224)
input_tensor = torch.Tensor(input_data)
out_torch = my_model(input_tensor)
out = inference(my_engine,4,input_data.astype(np.float32))

[05/24/2022-15:24:34] [TRT] [V] Using cublas as a tactic source
[05/24/2022-15:24:34] [TRT] [W] TensorRT was linked against cuBLAS/cuBLAS LT 11.8.0 but loaded cuBLAS/cuBLAS LT 11.5.1
[05/24/2022-15:24:34] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 4299, GPU 1215 (MiB)
[05/24/2022-15:24:34] [TRT] [V] Total per-runner device persistent memory is 105472
[05/24/2022-15:24:34] [TRT] [V] Total per-runner host persistent memory is 53200
[05/24/2022-15:24:34] [TRT] [V] Allocated activation device memory of size 21776384
[05/24/2022-15:24:34] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +21, now: CPU 0, GPU 57 (MiB)


In [28]:
out_torch

(tensor([[-0.8886,  0.7305,  1.5469,  ...,  1.3973, -0.2703,  1.5398],
         [ 0.4422, -1.4356, -0.4701,  ..., -1.0412, -0.9055, -1.7349],
         [-0.8149,  0.0101, -0.4138,  ..., -1.2335, -0.9594,  0.8930],
         [ 0.7111,  0.0948, -1.3353,  ...,  0.4502,  2.3650,  0.3567]],
        grad_fn=<AddmmBackward0>),
 tensor([[-1.4870,  1.2106,  1.1281,  ...,  0.1757,  0.0764,  3.5329],
         [ 1.1356,  0.3842,  0.9476,  ..., -0.5550, -0.9377, -1.5505],
         [-0.4595,  0.9924,  0.1319,  ..., -0.3796,  0.2248,  0.7724],
         [ 0.6558, -0.9540, -1.8382,  ...,  0.6027,  3.4104, -0.0396]],
        grad_fn=<AddmmBackward0>))